**IMPORTING LIBRARIES**

In [ ]:
import pandas as pd
import numpy as np
import re
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [ ]:
from gensim.models import Phrases


In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
!pip install pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 89.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 34.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.0.3 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.25.1 which is incompatible.
tensorflow 2.12.0 

In [ ]:
nltk.download('stopwords')
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

**DATASET HANDLING**

In [ ]:
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11314 entries, 0 to 11313
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   content       11314 non-null  object
 1   target        11314 non-null  int64 
 2   target_names  11314 non-null  object
dtypes: int64(1), object(2)
memory usage: 353.6+ KB


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
df=df[:300]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
data = df.content.values.tolist()

email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b'
data = [re.sub(email_pattern, '', sent) for sent in data]

# Remove newline characters
data = [re.sub(r'\r?\n', '', sent) for sent in data]


# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
tokenized_documents = []
for document in data:
    tokens = word_tokenize(document.lower())
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha() and not token in stop_words]
    tokenized_documents.append(tokens)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
tokenized_documents

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[['wheres',
  'thing',
  'subject',
  'car',
  'university',
  'maryland',
  'college',
  'parklines',
  'wondering',
  'anyone',
  'could',
  'enlighten',
  'car',
  'sawthe',
  'day',
  'sport',
  'car',
  'looked',
  'late',
  'called',
  'bricklin',
  'door',
  'really',
  'small',
  'addition',
  'front',
  'bumper',
  'separate',
  'rest',
  'body',
  'know',
  'anyone',
  'tellme',
  'model',
  'name',
  'engine',
  'spec',
  'yearsof',
  'production',
  'car',
  'made',
  'history',
  'whatever',
  'info',
  'youhave',
  'funky',
  'looking',
  'car',
  'please',
  'il',
  'brought',
  'neighborhood',
  'lerxst'],
 ['guy',
  'kuo',
  'subject',
  'si',
  'clock',
  'poll',
  'final',
  'callsummary',
  'final',
  'call',
  'si',
  'clock',
  'reportskeywords',
  'si',
  'acceleration',
  'clock',
  'university',
  'washingtonlines',
  'fair',
  'number',
  'brave',
  'soul',
  'upgraded',
  'si',
  'clock',
  'oscillator',
  'haveshared',
  'experience',
  'poll',
  'please',
 

In [ ]:
id2word = corpora.Dictionary(tokenized_documents)
texts = tokenized_documents
corpus = [id2word.doc2bow(text) for text in texts]
print(corpus)                     ###(wordid, frequency)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[[(0, 1), (1, 2), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 5), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1)], [(10, 2), (30, 2), (39, 1), (42, 1), (48, 1), (49, 1), (50, 2), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 5), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 2), (66, 1), (67, 2), (68, 2), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 2), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 2), (82, 1), (83, 1), (84, 1), (85, 1), (86, 4), (87, 1), (88, 1), (89, 2), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1)], [(9, 3), (18, 2), (19, 1), (23, 1), (32, 2), (39, 1), (42, 1), (45, 1), (67, 1), (70, 1

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=15,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, '0.031*"subject" + 0.014*"organization" + 0.014*"university" + 0.013*"would" + 0.013*"use" + 0.011*"need" + 0.011*"computer" + 0.009*"know" + 0.009*"like" + 0.008*"want"'), (1, '0.016*"weapon" + 0.009*"mass" + 0.008*"purchased" + 0.007*"picture" + 0.007*"sale" + 0.006*"article" + 0.006*"destruction" + 0.006*"say" + 0.006*"box" + 0.006*"contact"'), (2, '0.010*"earth" + 0.008*"reserve" + 0.008*"study" + 0.007*"titan" + 0.006*"possible" + 0.006*"image" + 0.005*"author" + 0.005*"united" + 0.005*"moral" + 0.005*"code"'), (3, '0.022*"god" + 0.010*"bike" + 0.006*"movie" + 0.006*"islamic" + 0.006*"driver" + 0.006*"think" + 0.005*"matthew" + 0.005*"say" + 0.005*"anyway" + 0.005*"knew"'), (4, '0.024*"window" + 0.011*"jim" + 0.011*"simms" + 0.010*"dan" + 0.009*"font" + 0.008*"software" + 0.008*"tower" + 0.007*"do" + 0.007*"microsoft" + 0.007*"cable"'), (5, '0.009*"armenian" + 0.009*"subject" + 0.007*"people" + 0.007*"article" + 0.007*"one" + 0.006*"writes" + 0.005*"could" + 0.005*"day" + 0.0

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
print('Perplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

coherence_model_lda = CoherenceModel(model=lda_model, texts=tokenized_documents, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Perplexity:  -10.102228643833968
Coherence Score:  0.4807246645767128


In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
10    -0.297716  0.047382       1        1  26.156747
12    -0.028790 -0.466314       2        1  16.091431
5     -0.195835  0.050169       3        1  16.079268
0     -0.218378  0.035306       4        1  14.101084
11     0.025306  0.038759       5        1   4.385621
3      0.002844  0.037912       6        1   4.084540
7      0.090764  0.014967       7        1   3.991565
8      0.041866  0.037666       8        1   3.174779
1      0.008395  0.042635       9        1   2.850619
2      0.112791  0.021980      10        1   2.690856
4      0.050902  0.038445      11        1   2.231288
6      0.127471 -0.001346      12        1   1.979657
14     0.086291  0.033898      13        1   1.250466
9      0.099279  0.032646      14        1   0.916956
13     0.094808  0.035895      15        1   0.015123, topic_info=         Term        Freq        Total Category  logprob  loglift
5940       ax  4798.00000  4798.000000  Default  30.0000  30.0000
2903      max   339.00000   339.000000  Default  29.0000  29.0000
3634        n   102.00000   102.000000  Default  28.0000  28.0000
3671        p    97.00000    97.000000  Default  27.0000  27.0000
1213        g   102.00000   102.000000  Default  26.0000  26.0000
...       ...         ...          ...      ...      ...      ...
513      mode     0.00062    15.162725  Topic15  -9.2486  -1.3075
4443  desonia     0.00062     2.890535  Topic15  -9.2486   0.3499
4456     plug     0.00062     6.655395  Topic15  -9.2486  -0.4841
3169  russian     0.00062    25.776678  Topic15  -9.2486  -1.8381
404       say     0.00062   108.487986  Topic15  -9.2486  -3.2753

[777 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
1261      6  0.749166    accident
1261      8  0.149833    accident
320       9  0.800159  accidental
3321     10  0.896101    accurate
3322      7  0.900411     achieve
...     ...       ...         ...
1069     13  0.191335        york
1776     11  0.947401       youll
6124      2  0.985613           z
2930      5  0.940201        zero
2721     13  0.561002         zod

[1157 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 13, 6, 1, 12, 4, 8, 9, 2, 3, 5, 7, 15, 10, 14])